<a href="https://colab.research.google.com/github/lfmartins/introduction-to-computational-mathematics/blob/main/17_data_analysis_with_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this notebook we explore `pandas`, a Python module designed for data manipulation and data analysis. The first thing we need to do is to import the data. There are different approaches to load data into `pandas`, depending on where the data is.

In this notebook, we will use the file `movies.csv`. The `csv` extension refers to *comma separated values*, which is a very basic text format for storing data. 

Run the following code cell to import the data:

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/lfmartins/introduction-to-computational-mathematics/main/movies.csv'
movies_df = pd.read_csv(url, index_col='Film')

In this example, the data is imported from GitHub, an online system designed for collaborative software development. This course is itself stored as a GitHub repository. It is very common today to have data stored in a web site.

The data is read with the function `pd.read_csv()` from the URL of the file in GitHub. It is also possible to read a local file by entering its name in place of the URL.

Let's now see how to work with the data in Pandas.



# Dataframes and Series

All data in `pandas` is stored in two kinds of objects:

- `DataFrame` represents a two-dimensional array (that is, a table) of data.
- `Series` represents a one-dimensional array of data.

A `DataFrame` is a collection of `Series` objects, representing the columns of the table.

The data for our example is stored in the variable `movies_df`. We can now examine the contents of the file:

In [ ]:
movies_df

Notice that `pandas`, by default, does not show the whole table. Data sets are usually large, so they can't be seen in their entirety.

Now click on the "magic wand" icon that appears under the data set. This will create an interactive display that allows "paging" on the data. 

Another useful method for seeing the structure of a `DataFrame` is `head()`:

In [ ]:
movies_df.head(5)

This displays the first five rows of the table.

To know the size of the data set, use the function `len()`:

In [ ]:
len(movies_df)

Data is organized in columns. To get the column names, we can use the variable `columns`:

In [ ]:
cols = movies_df.columns
cols

Notice that the first column does not appear in the list of columns. To access a column in the `DataFrame`, we use use the column name as a dictionary key:

In [ ]:
rtpercent = movies_df['Rotten Tomatoes %']
rtpercent

The first column in a `DataFrame` is called *index column*, and is used to identify rows in the dataset:

In [ ]:
movies_df.loc['When in Rome']

It is sometimes useful to reorder the dataset, which is accomplished with the method `sort_values`. Let's say we would like to identify the movies with highest audience score:

In [ ]:
movies_df.sort_values(by='Audience score %', ascending=False).head(10)

In this code, notice the use of a *pipeline*: we first call the method `sort_values()` of the object `movies_df`. Then, we call the method `head()` on the result to display the first 10 rows of the table. This kind of piplining is very common and useful. 

# Cleaning the Data

Data ofent does not come in a format that is ready for analysis. Data cleanup is usually the first task in a project.

Notice that the column `Genre` is somewhat inconsistent. Some of the entries are capitalized and others are not. Also notice in the output above that the genre for *Midnight in Paris* is incorrectly spelled as "Romence". We can see the a summary of the values in a column using the method `value_counts():

In [ ]:
movies_df.value_counts('Genre')

We can see that there are only a few errors, so we fix them "manually". For example, there is an incorrectly spelled entry `Comdy`. Let's first find the row in which the error appears:

In [ ]:
movies_df.loc[movies_df['Genre'] == 'Comdy']

Once we know the row with the mistake, it is easy to correct it:

In [ ]:
movies_df.loc['Made of Honor', 'Genre'] = 'Comedy'
movies_df.loc['Made of Honor', 'Genre']

**Exercise:** Fix the other mistakes in the `Genre` column.

After you fix all incorrect entries, run the `value_counts` method again to make sure everything is fine:

In [ ]:
movies_df.value_counts('Genre')

The column `Worldwide Gross` also presents a problem. We would like the dollar values to be numerical, but they are strings. One way to do this is to

In [ ]:
def string_to_float(v):
  return float(v[1:])
movies_df['Worldwide Gross']=movies_df['Worldwide Gross'].apply(string_to_float)
movies_df.head(5)

The values in the `Worldwide Gross` column are now of type `float`.

**Exercise:** Find the 10 highest grossing movies in the dataset.

In [ ]:
movies_df.sort_values(by='Worldwide Gross', ascending=False).head(10)

# Exploring the Data

Let's now start exploring the data. Let's start examining the `Worldwide Gross` column. We first construct a histogram of the data:

In [ ]:
movies_df['Worldwide Gross'].plot.hist()
None

We can see that the distribution is quite asymmetric, with most movies having low grossing values (a sobering fact for the movie industry).

We can get a summary of the data with the method `describe()`:



In [ ]:
movies_df['Worldwide Gross'].describe()

This yields some basic statistics for the `Worldwide Gross` column: the mean, standard deviation, minimum, quartiles and maximum.

The quartiles (25%, 50%, 75%)are points that split the data set in 4 equal segments, each containing 25% of the data. For example, 25% of the movies gross between 79.18 and 205.3 millions of dollars.

The 50% quartile is also the median of the data set, which is the point that splits the data into two segments, each containing half of the data. Notice that the median (79.18) is much smaller than the mean (145.217532). This indicates that the data has a long "tail" to the right.

This statistical summary has a nice representation called a *boxplot*:

In [ ]:
movies_df['Worldwide Gross'].plot(kind='box')
None

In this graph, the dots on top of the boxplot represent *outliers*, data values that are outside of the range where most of the data is. The box and segments (sometimes called "whiskers") represent, from top to bottom, the maximum, 75th percentile, 50th percentile (median), 25th percentile and minimum. For example, we can see that the top 25% grossing movies make between 200 and 400 millions of dollars.

Let's now dig a little deeper in the data. Suppose we want to know what studios gross the most.

To do this, we first extract a `DataFrame` that has only the two columns that interest us:

In [ ]:
studio_gross_df = movies_df[['Lead Studio', 'Worldwide Gross']]
studio_gross_df.head(5)

We now call the method `groupby` to find the total gross of each studio:

In [ ]:
studio_gross_df.groupby('Lead Studio').sum()

It is possible to pipeline the calls, so that we can run the code in a single line. In the code below, we do the following operations in succeccion: 

- Extract two columns from the data.
- Group by `Lead Studio`, computing the sum of `Worldwide Gross` for each group.
- Sort the table in descending order.

In [ ]:
movies_df[['Lead Studio', 'Worldwide Gross']].groupby('Lead Studio').sum().sort_values(by='Worldwide Gross', ascending=False)

Perhaps a better measure of the popularity of a studio's movies is the average value of `Worldwide Gross`:

In [ ]:
movies_df[['Lead Studio', 'Worldwide Gross']].groupby('Lead Studio').mean().sort_values(by='Worldwide Gross', ascending=False)

**Exercise:** Do an analysis of the `Profitability` column in the movie dataset.

# Linear Regression

One of the main goals of data analysis is to find relationships between variables. We might want to know, for example, if there is a relation between the Audience Score and the Rotten Tomatoes score of a movie.

A good starting point is to do a scatterplot of the data of interest:

In [ ]:
movies_df.plot.scatter(x='Rotten Tomatoes %', y='Audience score %')
None

The scatterplot indicates that there seems to be an increasing trend between the variables, but it is somewhat weak. To quantify this, we can use a *linear regression*, which finds the best linear relationship between the data.

Regression is an extensive subject, and `pandas` itself does not have methods to compute linear regression. Instead, we use the module `sklearn`, which is a library for machine learning. The code is in the following cell:

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
rt_scores = movies_df['Rotten Tomatoes %'].to_numpy().reshape(-1, 1)
a_scores = movies_df['Audience score %'].to_numpy()
model = LinearRegression().fit(rt_scores, a_scores)

We now need to extract the coefficients of the regression line:

In [ ]:
a = model.coef_[0]
b = model.intercept_
print(a, b)

From the output above, we get the following model:
\[
y = a x + b
\]
where $x$ represents the Rotten Tomatoes score and $y$ represents the Audience score.

Let's now construct a plot with the data and the linear model superimposed:

In [ ]:
import matplotlib.pyplot as plt
movies_df.plot.scatter(x='Rotten Tomatoes %', y='Audience score %')
xvalues = np.linspace(0, 100, 400)
yvalues = a * xvalues + b
plt.plot(xvalues, yvalues, lw=2, color='red')
None

This plot shows that there is an increasing relationship between the variables, but it is weak. To assess the quality of the model, we use the `score` model:

In [ ]:
model.score(rt_scores, a_scores)

This number is called R-squared, and measures the proportion of the variability in the data that is explained by the model. This is a number between 0 and 1, interpreted as a percent. A value of 37% shows that the relationship between the variables is weak.

Another interesting tool to explore correlations is a *scatter matrix*. The following code computes a scatter matrix for the quantitative variables in the movies dataset:

In [ ]:
from pandas.plotting import scatter_matrix
moviesn_df = movies_df[['Audience score %', 'Profitability',
       'Rotten Tomatoes %', 'Worldwide Gross']]
scatter_matrix(moviesn_df)
None

This matrix has scatterplots for all pairs of variables in the dataset, and histograms for each variable along the diagonal.

**Exercise** Investigate if there is a relationship between the Rotten Tomatoes scores and the Worldwide grossing of the movies in this dataset.